In [ ]:
# Step 1: Install necessary libraries
!pip install transformers datasets accelerate

# Step 2: Import required modules
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, pipeline

# Step 3: Load pretrained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 4: Define custom dataset
text_data = """The dragon’s shadow swept across ancient stone walls.
Beneath the silvered moon, the forest whispered forgotten spells.
A crown of thorns and stars adorned the exiled queen.
In the hidden valley, rivers glowed with liquid crystal light.
The sword sang softly, hungry for battle yet to come.
Old runes burned brighter as the prophecy drew near.
A lone mage traced circles of power into the midnight soil.
The castle’s gates stood silent, guarding secrets older than kings.
Phoenix ash scattered on the wind, rebirth woven into each ember.
Beyond the misty mountains, legends waited to wake once more.
"""

text_lines = text_data.strip().split('\n')

# Step 5: Create and tokenize dataset
dataset = Dataset.from_dict({"text": text_lines})

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Step 6: Set up training configuration
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    prediction_loss_only=True,
    fp16=True,
    report_to="none"
)

# Step 7: Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

# Step 8: Save the fine-tuned model and tokenizer
model.save_pretrained("gpt2-finetuned")
tokenizer.save_pretrained("gpt2-finetuned")

# Step 9: Generate text using the fine-tuned model
generator = pipeline("text-generation", model="gpt2-finetuned", tokenizer=tokenizer)
prompt = "Once upon a time"
output = generator(
    prompt,
    max_new_tokens=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.2
)

print(output[0]['generated_text'])
